Customized Visual Content Creation Tool

In [1]:
!pip install dotenv

!pip install langchain openai
!pip install langchain_openai
#!pip install langchain_google_genai
!pip install langchain_anthropic

!pip install gradio requests pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00


In [2]:
from dotenv                 import load_dotenv            as ld

from langchain_openai       import ChatOpenAI             as OAI
from langchain_anthropic    import ChatAnthropic          as CAnt

from langchain.schema       import AIMessage, HumanMessage, SystemMessage

import gradio as gr     # For creating the web UI
import requests         # For API calls to Hugging Face
from PIL import Image   # For handling image processing
from io import BytesIO  # For handling binary image data
import os               # For accessing environment variables
import openai           # For OpenAI API (Paid version, commented by default)


In [3]:
os.environ["HUGGINGFACE_API_KEY"] = "*******"
os.environ["OPENAI_API_KEY"]  = "*******"
os.environ["DEEPSEEK_API_KEY"] ="*******"

### Google Colab Setup used in this project for Stable Diffusion XL with Gradio
This is done to avoid exhausting HF GPU access.

In [4]:
# Set Hugging Face API key
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

# Function to generate image from text prompt
def generate_image(prompt):
    """Generates an image using Stable Diffusion from Hugging Face API."""
    try:
        API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0"
        headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

        response = requests.post(API_URL, headers=headers, json={"inputs": prompt})

        if response.status_code == 200 and response.headers["Content-Type"].startswith("image"):
            return Image.open(BytesIO(response.content))
        else:
            print(f"Error: API response invalid. Status Code: {response.status_code}")
            return Image.new("RGB", (512, 512), color="red")
    except Exception as e:
        print(f"Error: {e}")
        return Image.new("RGB", (512, 512), color="red")

# Define Gradio interface
interface = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(label="Enter your text prompt"),
    outputs=gr.Image(type="pil"),
    title="AI Image Generator (Colab Version)",
    description="Enter a text prompt to generate an image using Stable Diffusion XL."
)

# Launch Gradio app
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25ab7abe6215fae43c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Including OpenAI and DeepSeek R1 models alongside SAtabel-Diffusion

In [5]:
# Setting up API keys
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")            # OpenAI API key (Paid)
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")        # DeepSeek R1 API key

# Function to generate image using Stable Diffusion
def generate_image_stablediff(prompt):
    """Generates an image using Stable Diffusion from Hugging Face API."""
    try:
        API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0"
        headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

        response = requests.post(API_URL, headers=headers, json={"inputs": prompt})

        if response.status_code == 200 and response.headers["Content-Type"].startswith("image"):
            return Image.open(BytesIO(response.content))
        else:
            print(f"Error: API response invalid. Status Code: {response.status_code}")
            return Image.new("RGB", (512, 512), color="red")
    except Exception as e:
        print(f"Error: {e}")
        return Image.new("RGB", (512, 512), color="red")

# Function to generate image using DALL·E (Paid)
def generate_image_openai(prompt):
    """Generates an image using OpenAI's DALL·E model (Paid API)."""
    try:
        openai.api_key = OPENAI_API_KEY
        response = openai.Image.create(prompt=prompt, n=1, size="512x512")
        image_url = response["data"][0]["url"]
        return Image.open(BytesIO(requests.get(image_url).content))
    except Exception as e:
        print(f"Error: {e}")
        return Image.new("RGB", (512, 512), color="red")

# Function to generate image using DeepSeek R1
def generate_image_deepseek(prompt):
    """Generates an image using DeepSeek R1 API (Example placeholder)."""
    try:
        API_URL = "https://api.deepseek.com/v1/generate-image"
        headers = {"Authorization": f"Bearer {DEEPSEEK_API_KEY}"}
        response = requests.post(API_URL, headers=headers, json={"prompt": prompt})

        if response.status_code == 200 and response.headers["Content-Type"].startswith("image"):
            return Image.open(BytesIO(response.content))
        else:
            print(f"Error: API response invalid. Status Code: {response.status_code}")
            return Image.new("RGB", (512, 512), color="red")
    except Exception as e:
        print(f"Error: {e}")
        return Image.new("RGB", (512, 512), color="red")

# Define Gradio interface
interface = gr.Interface(
    fn=generate_image_stablediff,  # Default is set to Stable Diffusion
    inputs=gr.Textbox(label="Enter your text prompt"),
    outputs=gr.Image(type="pil"),
    title="AI Image Generator (Colab Version)",
    description="Enter a text prompt to generate an image using Stable Diffusion XL. Optional : Use OpenAI DALL·E or DeepSeek R1 functions in this code."
)

# Launch Gradio app
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e3e698d259ef1bc67.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
